In [1]:
import networkx as nx
import numpy as np
import os
import time
import pandas as pd
import matplotlib.pyplot as plt
from itertools import chain
from collections import defaultdict
from gensim.models import Word2Vec
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

C:\ProgramData\Anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


# Pipeline

### Data Loader

We need to load networks into memory. Usually networks are organized as pairs of nodes. And sometimes different edges have different weights. Hence, we use networkx.DiGraph to store such structure information and attributes.

In [2]:
def load_data(file_name):
    """
    read edges from an edge file
    """
    edges = list()
    df = pd.read_csv(file_name)
    for idx, row in df.iterrows():
        user_id, friends = row["user_id"], eval(row["friends"])
        for friend in friends:
            # add each friend relation as an edge
            edges.append((user_id, friend))
    edges = sorted(edges)
    
    return edges

def load_test_data(file_name):
    """
    read edges from an edge file
    """
    edges = list()
    scores = list()
    df = pd.read_csv(file_name)
    for idx, row in df.iterrows():
        edges.append((row["src"], row["dst"]))
    edges = sorted(edges)
    
    return edges

def generate_false_edges(true_edges, num_false_edges=5):
    """
    generate false edges given true edges
    """
    nodes = list(set(chain.from_iterable(true_edges)))
    N = len(nodes)
    true_edges = set(true_edges)
    print(N, len(true_edges))
    false_edges = set()
    
    while len(false_edges) < num_false_edges:
        # randomly sample two different nodes and check whether the pair exisit or not
        src, dst = nodes[int(np.random.rand() * N)], nodes[int(np.random.rand() * N)]
        if src != dst and (src, dst) not in true_edges and (src, dst) not in false_edges:
            false_edges.add((src, dst))
    false_edges = sorted(false_edges)
    
    return false_edges

def construct_graph_from_edges(edges):
    """
    generate a directed graph object given true edges
    DiGraph documentation: https://networkx.github.io/documentation/stable/reference/classes/digraph.html
    """
    # convert a list of edges {(u, v)} to a list of edges with weights {(u, v, w)}
    edge_weight = defaultdict(float)
    for e in edges:
        edge_weight[e] += 1.0
    weighed_edge_list = list()
    for e in sorted(edge_weight.keys()):
        weighed_edge_list.append((e[0], e[1], edge_weight[e]))
        
    graph = nx.DiGraph()
    graph.add_weighted_edges_from(weighed_edge_list)
    
    print("number of nodes:", graph.number_of_nodes())
    print("number of edges:", graph.number_of_edges())
    
    return graph

### Random Walk Generator

Random walk generators or random walkers yield random walks that contain both local and higher-order neighborhood information. However, naive non-uniform sampling is very slow, which requires O(n) time complexity. Here alias sampling can reduce the time complexity to O(1) with O(n) space. If you are interested, please see the following blog.

In [3]:
def alias_setup(probs):
    """
    compute utility lists for non-uniform sampling from discrete distributions.
    details: https://lips.cs.princeton.edu/the-alias-method-efficient-sampling-with-many-discrete-outcomes/
    """
    K = len(probs)
    q = np.zeros(K)
    J = np.zeros(K, dtype=np.int)

    smaller = list()
    larger = list()
    for kk, prob in enumerate(probs):
        q[kk] = K * prob
        if q[kk] < 1.0:
            smaller.append(kk)
        else:
            larger.append(kk)

    while len(smaller) > 0 and len(larger) > 0:
        small = smaller.pop()
        large = larger.pop()

        J[small] = large
        q[large] = q[large] + q[small] - 1.0
        if q[large] < 1.0:
            smaller.append(large)
        else:
            larger.append(large)

    return J, q

def get_alias_node(graph, node):
    """
    get the alias node setup lists for a given node.
    """
    # get the unnormalized probabilities with the first-order information
    unnormalized_probs = list()
    for nbr in graph.neighbors(node):
        unnormalized_probs.append(graph[node][nbr]["weight"])
    unnormalized_probs = np.array(unnormalized_probs)
    if len(unnormalized_probs) > 0:
        normalized_probs = unnormalized_probs / unnormalized_probs.sum()
    else:
        normalized_probs = unnormalized_probs
        
    return alias_setup(normalized_probs)
    
def get_alias_edge(graph, src, dst, p=1, q=1):
    """
    get the alias edge setup lists for a given edge.
    """
    # get the unnormalized probabilities with the second-order information
    unnormalized_probs = list()
    for dst_nbr in graph.neighbors(dst):
        if dst_nbr == src: # distance is 0
            unnormalized_probs.append(graph[dst][dst_nbr]["weight"]/p)
        elif graph.has_edge(dst_nbr, src): # distance is 1
            unnormalized_probs.append(graph[dst][dst_nbr]["weight"])
        else: # distance is 2
            unnormalized_probs.append(graph[dst][dst_nbr]["weight"]/q)
    unnormalized_probs = np.array(unnormalized_probs)
    if len(unnormalized_probs) > 0:
        normalized_probs = unnormalized_probs / unnormalized_probs.sum()
    else:
        normalized_probs = unnormalized_probs

    return alias_setup(normalized_probs)

def preprocess_transition_probs(graph, p=1, q=1):
    """
    preprocess transition probabilities for guiding the random walks.
    """
    alias_nodes = dict()
    for node in graph.nodes():
        alias_nodes[node] = get_alias_node(graph, node)

    alias_edges = dict()
    for edge in graph.edges():
        alias_edges[edge] = get_alias_edge(graph, edge[0], edge[1], p=p, q=q)

    return alias_nodes, alias_edges

The difference between DeepWalk and node2vec is how to generate random walks. The former only consider the first-order information while the latter also involves the second-order information.

In [4]:
def alias_draw(J, q):
    """
    draw sample from a non-uniform discrete distribution using alias sampling.
    """
    K = len(J)

    kk = int(np.floor(np.random.rand() * K))
    if np.random.rand() < q[kk]:
        return kk
    else:
        return J[kk]


# helper function to generate the long random walk as desired
def fallback(walk, fetch_last_num=1):
    if len(walk) > fetch_last_num:
        walk.pop()
        fetched = []
        for i in range(fetch_last_num):
            fetched.append(walk[-1-i])
        return walk, fetched
    else:
        return [], [None for _ in range(fetch_last_num)]

def generate_first_order_random_walk(graph, alias_nodes, 
                                     walk_length=10, start_node=None, verbose=False, max_trails=10):
    """
    simulate a random walk starting from start node and considering the first order information.
    max_trials: set the max trials to be one for standard random walk. Larger max_trails will make the generated biased.
    """
    if start_node == None:
        start_node = np.random.choice(graph.nodes())
    walk = [start_node]
    cur = start_node
    num_tried = 0
    
    ########## begin ##########
    while len(walk) < walk_length:
        cur_nbrs = list(graph.neighbors(cur))
        if len(cur_nbrs) > 0: # if we can sample next nodes
            # sample the next node based on alias_nodes
            cur = cur_nbrs[alias_draw(*alias_nodes[cur])]
            walk.append(cur)
        else: # if we can't do that
            num_tried += 1
            if num_tried >= max_trails:
                break

            walk, fetched = fallback(walk, fetch_last_num=1)
            cur = fetched[0]
            if len(walk) == 0: # if falls back to the empty walk
                start_node = np.random.choice(graph.nodes())
                walk = [start_node]
                cur = start_node
    ########## end ##########

    if verbose: 
        print(f'walk of lenght {len(walk)} generated with {num_tried} trails')
    return walk
    
def generate_second_order_random_walk(graph, alias_nodes, alias_edges, 
                                      walk_length=10, start_node=None, verbose=False, max_trails=10):
    """
    simulate a random walk starting from start node and considering the second order information.
    """
    if start_node == None:
        start_node = np.random.choice(graph.nodes())
    walk = [start_node]
    
    prev = None
    cur = start_node
    num_tried = 0

    ########## begin ##########
    while len(walk) < walk_length:
        cur_nbrs = list(graph.neighbors(cur))
        if len(cur_nbrs) > 0:
            if prev is None:
                # sample the next node based on alias_nodes
                prev, cur = cur, cur_nbrs[alias_draw(*alias_nodes[cur])]
            else:
                # sample the next node based on alias_edges
                prev, cur = cur, cur_nbrs[alias_draw(*alias_edges[(prev, cur)])]
            walk.append(cur)
        else:
            num_tried += 1
            if num_tried >= max_trails:
                break
            walk, (cur, prev) = fallback(walk, fetch_last_num=2)
            if len(walk) == 0:
                start_node = np.random.choice(graph.nodes())
                walk = [start_node]
                cur = start_node
                prev = None
    ########## end ##########
    if verbose: 
        print(f'walk of lenght {len(walk)} generated with {num_tried} trails')
    return walk

### Network Embedding Algorithms

In [5]:
def build_deepwalk(graph, alias_nodes, node_dim=10, num_walks=10, walk_length=10):
    """
    build a deepwalk model
    """
    print("building a DeepWalk model...", end="\t")
    st = time.time()
    np.random.seed(0)
    nodes = list(graph.nodes())
    walks = list()
    # generate random walks
    for walk_iter in range(num_walks):
        np.random.shuffle(nodes)
        for node in nodes:
            walks.append(generate_first_order_random_walk(
                graph, alias_nodes, walk_length=walk_length, start_node=node))
        
    walk_lens = [len(w) for w in walks]
    if len(walk_lens) > 0:
        avg_walk_len = sum(walk_lens) / len(walk_lens)
    else:
        avg_walk_len = 0.0
    print("number of walks: %d\taverage walk length: %.4f" % (len(walks), avg_walk_len), end="\t")
    
    # train a skip-gram model for these walks
    model = Word2Vec(walks, vector_size=node_dim, window=3, min_count=0, sg=1, workers=os.cpu_count(), epochs=10)
    print("training time: %.4f" % (time.time()-st))
    
    return model

def build_node2vec(graph, alias_nodes, alias_edges, node_dim=10, num_walks=10, walk_length=10):
    """
    build a node2vec model
    """
    print("building a node2vec model...", end="\t")
    st = time.time()
    np.random.seed(0)
    nodes = list(graph.nodes())
    walks = list()
    # generate random walks
    for walk_iter in range(num_walks):
        np.random.shuffle(nodes)
        for node in nodes:
            walks.append(generate_second_order_random_walk(
                graph, alias_nodes, alias_edges, walk_length=walk_length, start_node=node))
            
    walk_lens = [len(w) for w in walks]
    if len(walk_lens) > 0:
        avg_walk_len = sum(walk_lens) / len(walk_lens)
    else:
        avg_walk_len = 0.0    
    print("number of walks: %d\taverage walk length: %.4f" % (len(walks), avg_walk_len), end="\t")
    
    # train a skip-gram model for these walks
    model = Word2Vec(walks, vector_size=node_dim, window=3, min_count=0, sg=1, workers=os.cpu_count(), epochs=10)
    print("training time: %.4f" % (time.time()-st))
    
    return model

### Scorer

In [6]:
def get_cosine_sim(model, u, v):
    """
    get the cosine similarity between two nodes
    """
    try:
        u = model.wv[u]
        v = model.wv[v]
        return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))
    except:
        return 0.5

def get_auc_score(model, true_edges, false_edges):
    """
    get the auc score
    """
    y_true = [1] * len(true_edges) + [0] * len(false_edges)
    
    y_score = list()
    for e in true_edges:
        y_score.append(get_cosine_sim(model, e[0], e[1]))
    for e in false_edges:
        y_score.append(get_cosine_sim(model, e[0], e[1]))
    
    return roc_auc_score(y_true, y_score)

def write_pred(file_name, edges, scores):
    df = pd.DataFrame()
    df["src"] = [e[0] for e in edges]
    df["dst"] = [e[1] for e in edges]
    df["score"] = scores
    df.to_csv(file_name, index=False)
    
    



def write_valid_ans(file_name, edges, scores):
    df = pd.DataFrame()
    df["src"] = [e[0] for e in edges]
    df["dst"] = [e[1] for e in edges]
    df["score"] = scores
    df.to_csv(file_name, index=False)

### Try them over a Real-life Network

Firstly, we need to load edges into memory and use the networkx.DiGraph structure to store the graph.

In [7]:
train_file = "data/train.csv"
valid_file = "data/valid.csv"
test_file = "data/test.csv"

np.random.seed(0)
train_edges = load_data(train_file)
graph = construct_graph_from_edges(train_edges)
valid_edges = load_data(valid_file)
false_edges = generate_false_edges(train_edges+valid_edges, 40000-len(valid_edges))
test_edges = load_test_data(test_file)

number of nodes: 8328
number of edges: 100000
8474 119268


And we can build a DeepWalk model and a node2vec model. Here we set p=q=0.5 so that the walker will not go very far away from the start node.

In [8]:
alias_nodes, alias_edges = preprocess_transition_probs(graph, p=1, q=1)

In [9]:
model = build_deepwalk(graph, alias_nodes, node_dim=10, num_walks=10, walk_length=10)

building a DeepWalk model...	number of walks: 83280	average walk length: 9.8980	training time: 14.0067


In [10]:
model = build_node2vec(graph, alias_nodes, alias_edges, node_dim=10, num_walks=10, walk_length=10)

building a node2vec model...	number of walks: 83280	average walk length: 9.9910	training time: 16.3254


# Link Prediction

We can train the model with different parameters and test the model on the validation set. Please show your parameter search process and analysis in your code, and visualization tools (Heatmap, etc.) are recommended to make your analysis clear.

In [11]:
train_file = "data/train.csv"
valid_file = "data/valid.csv"
test_file = "data/test.csv"

np.random.seed(0)
train_edges = load_data(train_file)
graph = construct_graph_from_edges(train_edges)
valid_edges = load_data(valid_file)
false_edges = generate_false_edges(train_edges+valid_edges, 40000-len(valid_edges))
test_edges = load_test_data(test_file)

number of nodes: 8328
number of edges: 100000
8474 119268


# Parameter Training - Without Cross Validation

###  Deepwalk

In [13]:
np.random.seed(0)

node_dim = [5, 10, 15, 20, 25, 30]
num_walks = [5, 10, 20,30, 40]
walk_length = [5, 10, 20, 30, 40]
p_val = [0.25, 0.5, 0.75, 1, 1.25]
q_val = [0.25, 0.5, 0.75, 1, 1.25]

deepwalk_auc_scores = dict()



for p in p_val:
    for q in q_val:
        for dim in node_dim:
            for walks in num_walks:
                for length in walk_length:
                    alias_nodes, alias_edges = preprocess_transition_probs(graph, p=p, q=q)
                    print("node dim: %d,\tnum_walks: %d,\twalk_length: %d,\tp: %.2f,\tq: %.2f" % (
                            dim, walks, length, p, q), end="\t")
                    model = build_deepwalk(graph, alias_nodes, 
                                   node_dim=dim, num_walks=walks, walk_length=length)
                    deepwalk_auc_scores[(p,q,dim, walks, length)] = get_auc_score(model, valid_edges, false_edges)
                    print("auc: %.4f" % (deepwalk_auc_scores[(p,q,dim, walks, length)]))

node dim: 5,	num_walks: 5,	walk_length: 5,	p: 0.25,	q: 0.25	building a DeepWalk model...	number of walks: 41640	average walk length: 4.9622	training time: 6.9888
auc: 0.8764
node dim: 5,	num_walks: 5,	walk_length: 10,	p: 0.25,	q: 0.25	building a DeepWalk model...	number of walks: 41640	average walk length: 9.8976	training time: 10.2654
auc: 0.9081
node dim: 5,	num_walks: 5,	walk_length: 20,	p: 0.25,	q: 0.25	building a DeepWalk model...	number of walks: 41640	average walk length: 19.7535	training time: 15.7280
auc: 0.9163
node dim: 5,	num_walks: 5,	walk_length: 30,	p: 0.25,	q: 0.25	building a DeepWalk model...	number of walks: 41640	average walk length: 29.6132	training time: 17.3624
auc: 0.9175
node dim: 5,	num_walks: 5,	walk_length: 40,	p: 0.25,	q: 0.25	building a DeepWalk model...	number of walks: 41640	average walk length: 39.4357	training time: 22.4484
auc: 0.9195
node dim: 5,	num_walks: 10,	walk_length: 5,	p: 0.25,	q: 0.25	building a DeepWalk model...	number of walks: 83280	averag

node dim: 10,	num_walks: 40,	walk_length: 10,	p: 0.25,	q: 0.25	building a DeepWalk model...	number of walks: 333120	average walk length: 9.8984	training time: 63.7857
auc: 0.9326
node dim: 10,	num_walks: 40,	walk_length: 20,	p: 0.25,	q: 0.25	building a DeepWalk model...	number of walks: 333120	average walk length: 19.7608	training time: 102.0986
auc: 0.9303
node dim: 10,	num_walks: 40,	walk_length: 30,	p: 0.25,	q: 0.25	building a DeepWalk model...	number of walks: 333120	average walk length: 29.6068	training time: 125.6347
auc: 0.9286
node dim: 10,	num_walks: 40,	walk_length: 40,	p: 0.25,	q: 0.25	building a DeepWalk model...	number of walks: 333120	average walk length: 39.4376	training time: 172.9143
auc: 0.9273
node dim: 15,	num_walks: 5,	walk_length: 5,	p: 0.25,	q: 0.25	building a DeepWalk model...	number of walks: 41640	average walk length: 4.9622	training time: 5.1809
auc: 0.8371
node dim: 15,	num_walks: 5,	walk_length: 10,	p: 0.25,	q: 0.25	building a DeepWalk model...	number of wa

auc: 0.9286
node dim: 20,	num_walks: 30,	walk_length: 20,	p: 0.25,	q: 0.25	building a DeepWalk model...	number of walks: 249840	average walk length: 19.7612	training time: 71.9123
auc: 0.9260
node dim: 20,	num_walks: 30,	walk_length: 30,	p: 0.25,	q: 0.25	building a DeepWalk model...	number of walks: 249840	average walk length: 29.6069	training time: 101.1220
auc: 0.9239
node dim: 20,	num_walks: 30,	walk_length: 40,	p: 0.25,	q: 0.25	building a DeepWalk model...	number of walks: 249840	average walk length: 39.4396	training time: 128.3790
auc: 0.9224
node dim: 20,	num_walks: 40,	walk_length: 5,	p: 0.25,	q: 0.25	building a DeepWalk model...	number of walks: 333120	average walk length: 4.9633	training time: 40.0644
auc: 0.9201
node dim: 20,	num_walks: 40,	walk_length: 10,	p: 0.25,	q: 0.25	building a DeepWalk model...	number of walks: 333120	average walk length: 9.8984	training time: 57.9724
auc: 0.9300
node dim: 20,	num_walks: 40,	walk_length: 20,	p: 0.25,	q: 0.25	building a DeepWalk model.

node dim: 10,	num_walks: 10,	walk_length: 30,	p: 0.25,	q: 0.50	building a DeepWalk model...	number of walks: 83280	average walk length: 29.6058	training time: 30.4538
auc: 0.9317
node dim: 10,	num_walks: 10,	walk_length: 40,	p: 0.25,	q: 0.50	building a DeepWalk model...	number of walks: 83280	average walk length: 39.4396	training time: 38.6918
auc: 0.9308
node dim: 10,	num_walks: 20,	walk_length: 5,	p: 0.25,	q: 0.50	building a DeepWalk model...	number of walks: 166560	average walk length: 4.9630	training time: 19.8741
auc: 0.9238
node dim: 10,	num_walks: 20,	walk_length: 10,	p: 0.25,	q: 0.50	building a DeepWalk model...	number of walks: 166560	average walk length: 9.8981	training time: 27.7386
auc: 0.9325
node dim: 10,	num_walks: 20,	walk_length: 20,	p: 0.25,	q: 0.50	building a DeepWalk model...	number of walks: 166560	average walk length: 19.7587	training time: 44.3189
auc: 0.9320
node dim: 10,	num_walks: 20,	walk_length: 30,	p: 0.25,	q: 0.50	building a DeepWalk model...	number of wal

node dim: 30,	num_walks: 40,	walk_length: 30,	p: 0.25,	q: 0.50	building a DeepWalk model...	number of walks: 333120	average walk length: 29.6068	training time: 145.7796
auc: 0.9145
node dim: 30,	num_walks: 40,	walk_length: 40,	p: 0.25,	q: 0.50	building a DeepWalk model...	number of walks: 333120	average walk length: 39.4376	training time: 187.4314
auc: 0.9121
node dim: 40,	num_walks: 5,	walk_length: 5,	p: 0.25,	q: 0.50	building a DeepWalk model...	number of walks: 41640	average walk length: 4.9622	training time: 5.3189
auc: 0.7946
node dim: 40,	num_walks: 5,	walk_length: 10,	p: 0.25,	q: 0.50	building a DeepWalk model...	number of walks: 41640	average walk length: 9.8976	training time: 7.6779
auc: 0.8058
node dim: 40,	num_walks: 5,	walk_length: 20,	p: 0.25,	q: 0.50	building a DeepWalk model...	number of walks: 41640	average walk length: 19.7535	training time: 12.4034
auc: 0.8544
node dim: 40,	num_walks: 5,	walk_length: 30,	p: 0.25,	q: 0.50	building a DeepWalk model...	number of walks: 4

...

auc: 0.9142
node dim: 5,	num_walks: 20,	walk_length: 30,	p: 1.25,	q: 1.00	building a DeepWalk model...	number of walks: 166560	average walk length: 29.6095	training time: 57.1466
auc: 0.9183
node dim: 5,	num_walks: 20,	walk_length: 40,	p: 1.25,	q: 1.00	building a DeepWalk model...	number of walks: 166560	average walk length: 39.4343	training time: 72.5282
auc: 0.9163
node dim: 5,	num_walks: 30,	walk_length: 5,	p: 1.25,	q: 1.00	building a DeepWalk model...	number of walks: 249840	average walk length: 4.9632	training time: 28.5495
auc: 0.9095
node dim: 5,	num_walks: 30,	walk_length: 10,	p: 1.25,	q: 1.00	building a DeepWalk model...	number of walks: 249840	average walk length: 9.8983	training time: 40.0351
auc: 0.9159
node dim: 5,	num_walks: 30,	walk_length: 20,	p: 1.25,	q: 1.00	building a DeepWalk model...	number of walks: 249840	average walk length: 19.7612	training time: 62.4010
auc: 0.9164
node dim: 5,	num_walks: 30,	walk_length: 30,	p: 1.25,	q: 1.00	building a DeepWalk model...	numbe

auc: 0.9305
node dim: 15,	num_walks: 10,	walk_length: 40,	p: 1.25,	q: 1.00	building a DeepWalk model...	number of walks: 83280	average walk length: 39.4396	training time: 40.5832
auc: 0.9290
node dim: 15,	num_walks: 20,	walk_length: 5,	p: 1.25,	q: 1.00	building a DeepWalk model...	number of walks: 166560	average walk length: 4.9630	training time: 19.5705
auc: 0.9170
node dim: 15,	num_walks: 20,	walk_length: 10,	p: 1.25,	q: 1.00	building a DeepWalk model...	number of walks: 166560	average walk length: 9.8981	training time: 28.1606
auc: 0.9316
node dim: 15,	num_walks: 20,	walk_length: 20,	p: 1.25,	q: 1.00	building a DeepWalk model...	number of walks: 166560	average walk length: 19.7587	training time: 45.3688
auc: 0.9317
node dim: 15,	num_walks: 20,	walk_length: 30,	p: 1.25,	q: 1.00	building a DeepWalk model...	number of walks: 166560	average walk length: 29.6095	training time: 63.3929
auc: 0.9289
node dim: 15,	num_walks: 20,	walk_length: 40,	p: 1.25,	q: 1.00	building a DeepWalk model...	

node dim: 40,	num_walks: 40,	walk_length: 40,	p: 1.25,	q: 1.00	building a DeepWalk model...	number of walks: 333120	average walk length: 39.4376	training time: 167.8773
auc: 0.9023
node dim: 5,	num_walks: 5,	walk_length: 5,	p: 1.25,	q: 1.25	building a DeepWalk model...	number of walks: 41640	average walk length: 4.9622	training time: 5.0179
auc: 0.8779
node dim: 5,	num_walks: 5,	walk_length: 10,	p: 1.25,	q: 1.25	building a DeepWalk model...	number of walks: 41640	average walk length: 9.8976	training time: 6.9077
auc: 0.9086
node dim: 5,	num_walks: 5,	walk_length: 20,	p: 1.25,	q: 1.25	building a DeepWalk model...	number of walks: 41640	average walk length: 19.7535	training time: 10.7044
auc: 0.9162
node dim: 5,	num_walks: 5,	walk_length: 30,	p: 1.25,	q: 1.25	building a DeepWalk model...	number of walks: 41640	average walk length: 29.6132	training time: 14.7384
auc: 0.9177
node dim: 5,	num_walks: 5,	walk_length: 40,	p: 1.25,	q: 1.25	building a DeepWalk model...	number of walks: 41640	ave

node dim: 10,	num_walks: 40,	walk_length: 5,	p: 1.25,	q: 1.25	building a DeepWalk model...	number of walks: 333120	average walk length: 4.9633	training time: 38.6134
auc: 0.9306
node dim: 10,	num_walks: 40,	walk_length: 10,	p: 1.25,	q: 1.25	building a DeepWalk model...	number of walks: 333120	average walk length: 9.8984	training time: 58.7480
auc: 0.9328
node dim: 10,	num_walks: 40,	walk_length: 20,	p: 1.25,	q: 1.25	building a DeepWalk model...	number of walks: 333120	average walk length: 19.7608	training time: 88.1024
auc: 0.9300
node dim: 10,	num_walks: 40,	walk_length: 30,	p: 1.25,	q: 1.25	building a DeepWalk model...	number of walks: 333120	average walk length: 29.6068	training time: 120.9287
auc: 0.9287
node dim: 10,	num_walks: 40,	walk_length: 40,	p: 1.25,	q: 1.25	building a DeepWalk model...	number of walks: 333120	average walk length: 39.4376	training time: 153.6911
auc: 0.9268
node dim: 15,	num_walks: 5,	walk_length: 5,	p: 1.25,	q: 1.25	building a DeepWalk model...	number of w

auc: 0.9149
node dim: 20,	num_walks: 30,	walk_length: 10,	p: 1.25,	q: 1.25	building a DeepWalk model...	number of walks: 249840	average walk length: 9.8983	training time: 43.3925
auc: 0.9279
node dim: 20,	num_walks: 30,	walk_length: 20,	p: 1.25,	q: 1.25	building a DeepWalk model...	number of walks: 249840	average walk length: 19.7612	training time: 70.4504
auc: 0.9268
node dim: 20,	num_walks: 30,	walk_length: 30,	p: 1.25,	q: 1.25	building a DeepWalk model...	number of walks: 249840	average walk length: 29.6069	training time: 98.7178
auc: 0.9245
node dim: 20,	num_walks: 30,	walk_length: 40,	p: 1.25,	q: 1.25	building a DeepWalk model...	number of walks: 249840	average walk length: 39.4396	training time: 126.1072
auc: 0.9221
node dim: 20,	num_walks: 40,	walk_length: 5,	p: 1.25,	q: 1.25	building a DeepWalk model...	number of walks: 333120	average walk length: 4.9633	training time: 41.1505
auc: 0.9213
node dim: 20,	num_walks: 40,	walk_length: 10,	p: 1.25,	q: 1.25	building a DeepWalk model..

### Store the testing result to csv

In [15]:
deepwalk = pd.DataFrame()
deepwalk['keys'] = deepwalk_auc_scores.keys()
deepwalk['items'] = deepwalk_auc_scores.values()

list_for_p = []
list_for_q = []
list_for_num_dim = []
list_for_num_walks = []
list_for_walk_length = []
for i in range(3750):
    list_for_p.append(deepwalk.iloc[i][0][0])
    list_for_q.append(deepwalk.iloc[i][0][1])
    list_for_num_dim.append(deepwalk.iloc[i][0][2])
    list_for_num_walks.append(deepwalk.iloc[i][0][3])
    list_for_walk_length.append(deepwalk.iloc[i][0][4])
    
deepwalk['p'] = list_for_p
deepwalk['q'] = list_for_q 
deepwalk['node_dim'] = list_for_num_dim
deepwalk['num_walks'] = list_for_num_walks
deepwalk['walk_length'] = list_for_walk_length

deepwalk.to_csv('deepwalk.csv')

###  Node2vec

#### Tested on different notebook separately

# Parameter Training - Cross Validation

#### Function for parameter search

In [16]:
def casedeepwalk(kfold=5, node_dim=10, num_walks=10, walk_length=5, p=0.5, q=0.5):
    np.random.seed(0)

    deepwalk_auc_scores = list()
    kf = KFold(n_splits=kfold, shuffle=True)
    for k, (train_idx, valid_idx) in enumerate(kf.split(train_edges)):
        # split edges into training and validation
        cv_train_edges = [train_edges[idx] for idx in train_idx]
        cv_valid_edges = [train_edges[idx] for idx in valid_idx]
        # generate the same validation size of false edges
        cv_false_edges = generate_false_edges(train_edges, num_false_edges=len(cv_valid_edges))

        # construct the graph and preprocess transition probabilities
        cv_graph = construct_graph_from_edges(cv_train_edges)
        alias_nodes, alias_edges = preprocess_transition_probs(cv_graph, p=p, q=q)

        # build models and get auc scores
        model = build_deepwalk(cv_graph, alias_nodes,
                               node_dim=node_dim, num_walks=num_walks, walk_length=walk_length)
        deepwalk_auc_scores.append(get_auc_score(model, cv_valid_edges, cv_false_edges))

        
    deepwalk_auc_scores = np.array(deepwalk_auc_scores)
    
    print("DeepWalk: avg auc score: %.4f\tstd: %.4f" % (deepwalk_auc_scores.mean(), deepwalk_auc_scores.std()))

In [17]:
def casenode2vec(kfold=5, node_dim=10, num_walks=10, walk_length=5, p=0.5, q=0.5):
    np.random.seed(0)

    node2vec_auc_scores = list()
    kf = KFold(n_splits=kfold, shuffle=True)
    for k, (train_idx, valid_idx) in enumerate(kf.split(train_edges)):
        # split edges into training and validation
        cv_train_edges = [train_edges[idx] for idx in train_idx]
        cv_valid_edges = [train_edges[idx] for idx in valid_idx]
        # generate the same validation size of false edges
        cv_false_edges = generate_false_edges(train_edges, num_false_edges=len(cv_valid_edges))

        # construct the graph and preprocess transition probabilities
        cv_graph = construct_graph_from_edges(cv_train_edges)
        alias_nodes, alias_edges = preprocess_transition_probs(cv_graph, p=p, q=q)

        # build models and get auc scores
        
        model = build_node2vec(cv_graph, alias_nodes, alias_edges,
                               node_dim=node_dim, num_walks=num_walks, walk_length=walk_length)
        node2vec_auc_scores.append(get_auc_score(model, cv_valid_edges, cv_false_edges))

    
    node2vec_auc_scores = np.array(node2vec_auc_scores)
    
    print("node2vec: avg auc score: %.4f\tstd: %.4f" % (node2vec_auc_scores.mean(), node2vec_auc_scores.std()))

### Deepwalk

In [18]:
deepwalk = pd.read_csv('deepwalk.csv')

In [19]:
deepwalk = deepwalk.sort_values(by=['items'],ascending= False)

In [32]:
%%time
for i in range(60):
        casedeepwalk(kfold=5, node_dim=result.iloc[i][5], num_walks=result.iloc[i][6], walk_length=result.iloc[i][7], p=result.iloc[i][3], q=result.iloc[i][4])

8328 100000
number of nodes: 8029
number of edges: 80000
building a DeepWalk model...	number of walks: 240870	average walk length: 9.8271	training time: 52.4521
8328 100000
number of nodes: 8060
number of edges: 80000
building a DeepWalk model...	number of walks: 241800	average walk length: 9.8136	training time: 52.2801
8328 100000
number of nodes: 8028
number of edges: 80000
building a DeepWalk model...	number of walks: 240840	average walk length: 9.8093	training time: 55.0604
8328 100000
number of nodes: 8052
number of edges: 80000
building a DeepWalk model...	number of walks: 241560	average walk length: 9.8094	training time: 61.0218
8328 100000
number of nodes: 8058
number of edges: 80000
building a DeepWalk model...	number of walks: 241740	average walk length: 9.8233	training time: 57.5242
DeepWalk: avg auc score: 0.9242	std: 0.0010
8328 100000
number of nodes: 8029
number of edges: 80000
building a DeepWalk model...	number of walks: 240870	average walk length: 9.8271	training time

building a DeepWalk model...	number of walks: 322400	average walk length: 9.8141	training time: 67.8803
8328 100000
number of nodes: 8028
number of edges: 80000
building a DeepWalk model...	number of walks: 321120	average walk length: 9.8091	training time: 68.4503
8328 100000
number of nodes: 8052
number of edges: 80000
building a DeepWalk model...	number of walks: 322080	average walk length: 9.8083	training time: 68.3812
8328 100000
number of nodes: 8058
number of edges: 80000
building a DeepWalk model...	number of walks: 322320	average walk length: 9.8229	training time: 66.1508
DeepWalk: avg auc score: 0.9240	std: 0.0012
8328 100000
number of nodes: 8029
number of edges: 80000
building a DeepWalk model...	number of walks: 80290	average walk length: 29.2654	training time: 33.7488
8328 100000
number of nodes: 8060
number of edges: 80000
building a DeepWalk model...	number of walks: 80600	average walk length: 29.2253	training time: 34.2490
8328 100000
number of nodes: 8028
number of edg

building a DeepWalk model...	number of walks: 80280	average walk length: 29.2203	training time: 33.8616
8328 100000
number of nodes: 8052
number of edges: 80000
building a DeepWalk model...	number of walks: 80520	average walk length: 29.2259	training time: 34.2502
8328 100000
number of nodes: 8058
number of edges: 80000
building a DeepWalk model...	number of walks: 80580	average walk length: 29.3079	training time: 33.5548
DeepWalk: avg auc score: 0.9216	std: 0.0010
8328 100000
number of nodes: 8029
number of edges: 80000
building a DeepWalk model...	number of walks: 80290	average walk length: 19.5702	training time: 25.2669
8328 100000
number of nodes: 8060
number of edges: 80000
building a DeepWalk model...	number of walks: 80600	average walk length: 19.5423	training time: 25.8275
8328 100000
number of nodes: 8028
number of edges: 80000
building a DeepWalk model...	number of walks: 80280	average walk length: 19.5389	training time: 25.8641
8328 100000
number of nodes: 8052
number of edg

building a DeepWalk model...	number of walks: 322080	average walk length: 9.8083	training time: 69.1308
8328 100000
number of nodes: 8058
number of edges: 80000
building a DeepWalk model...	number of walks: 322320	average walk length: 9.8229	training time: 66.0269
DeepWalk: avg auc score: 0.9239	std: 0.0012
8328 100000
number of nodes: 8029
number of edges: 80000
building a DeepWalk model...	number of walks: 160580	average walk length: 9.8268	training time: 33.5330
8328 100000
number of nodes: 8060
number of edges: 80000
building a DeepWalk model...	number of walks: 161200	average walk length: 9.8133	training time: 34.6863
8328 100000
number of nodes: 8028
number of edges: 80000
building a DeepWalk model...	number of walks: 160560	average walk length: 9.8091	training time: 35.2088
8328 100000
number of nodes: 8052
number of edges: 80000
building a DeepWalk model...	number of walks: 161040	average walk length: 9.8096	training time: 34.8303
8328 100000
number of nodes: 8058
number of edg

### Node2vec

In [56]:
result = pd.read_excel('result.xlsx',sheet_name='Sheet3')

In [58]:
node2vec = result[result['Model'] == "Node2vec"] 

In [59]:
node2vec = node2vec.sort_values(by=['items'],ascending= False)

In [62]:
%%time
for i in range(60):
        casenode2vec(kfold=5, node_dim=node2vec.iloc[i][5], num_walks=node2vec.iloc[i][6], walk_length=node2vec.iloc[i][7], p=node2vec.iloc[i][3], q=node2vec.iloc[i][4])

8328 100000
number of nodes: 8029
number of edges: 80000
building a node2vec model...	number of walks: 80290	average walk length: 19.9191	training time: 30.6472
8328 100000
number of nodes: 8060
number of edges: 80000
building a node2vec model...	number of walks: 80600	average walk length: 19.9266	training time: 30.9239
8328 100000
number of nodes: 8028
number of edges: 80000
building a node2vec model...	number of walks: 80280	average walk length: 19.9406	training time: 35.1904
8328 100000
number of nodes: 8052
number of edges: 80000
building a node2vec model...	number of walks: 80520	average walk length: 19.9301	training time: 31.9827
8328 100000
number of nodes: 8058
number of edges: 80000
building a node2vec model...	number of walks: 80580	average walk length: 19.9579	training time: 33.0686
node2vec: avg auc score: 0.9214	std: 0.0015
8328 100000
number of nodes: 8029
number of edges: 80000
building a node2vec model...	number of walks: 80290	average walk length: 19.9090	training time

building a node2vec model...	number of walks: 161200	average walk length: 9.9798	training time: 80.8190
8328 100000
number of nodes: 8028
number of edges: 80000
building a node2vec model...	number of walks: 160560	average walk length: 9.9809	training time: 84.6953
8328 100000
number of nodes: 8052
number of edges: 80000
building a node2vec model...	number of walks: 161040	average walk length: 9.9776	training time: 69.6494
8328 100000
number of nodes: 8058
number of edges: 80000
building a node2vec model...	number of walks: 161160	average walk length: 9.9854	training time: 64.0466
node2vec: avg auc score: 0.9207	std: 0.0011
8328 100000
number of nodes: 8029
number of edges: 80000
building a node2vec model...	number of walks: 160580	average walk length: 9.9774	training time: 80.7874
8328 100000
number of nodes: 8060
number of edges: 80000
building a node2vec model...	number of walks: 161200	average walk length: 9.9798	training time: 74.5701
8328 100000
number of nodes: 8028
number of edg

building a node2vec model...	number of walks: 80280	average walk length: 29.7501	training time: 41.1375
8328 100000
number of nodes: 8052
number of edges: 80000
building a node2vec model...	number of walks: 80520	average walk length: 29.7121	training time: 41.1620
8328 100000
number of nodes: 8058
number of edges: 80000
building a node2vec model...	number of walks: 80580	average walk length: 29.8139	training time: 40.7389
node2vec: avg auc score: 0.9208	std: 0.0010
8328 100000
number of nodes: 8029
number of edges: 80000
building a node2vec model...	number of walks: 160580	average walk length: 9.9784	training time: 41.7514
8328 100000
number of nodes: 8060
number of edges: 80000
building a node2vec model...	number of walks: 161200	average walk length: 9.9801	training time: 43.7170
8328 100000
number of nodes: 8028
number of edges: 80000
building a node2vec model...	number of walks: 160560	average walk length: 9.9832	training time: 43.6270
8328 100000
number of nodes: 8052
number of edg

building a node2vec model...	number of walks: 161040	average walk length: 9.9778	training time: 49.4879
8328 100000
number of nodes: 8058
number of edges: 80000
building a node2vec model...	number of walks: 161160	average walk length: 9.9872	training time: 46.8295
node2vec: avg auc score: 0.9213	std: 0.0014
8328 100000
number of nodes: 8029
number of edges: 80000
building a node2vec model...	number of walks: 80290	average walk length: 19.8656	training time: 32.8827
8328 100000
number of nodes: 8060
number of edges: 80000
building a node2vec model...	number of walks: 80600	average walk length: 19.8685	training time: 33.3153
8328 100000
number of nodes: 8028
number of edges: 80000
building a node2vec model...	number of walks: 80280	average walk length: 19.8754	training time: 33.7539
8328 100000
number of nodes: 8052
number of edges: 80000
building a node2vec model...	number of walks: 80520	average walk length: 19.8702	training time: 34.0639
8328 100000
number of nodes: 8058
number of edg

# Conclusion
### The best model is Deepwalk with parameters:
### "P" = 1
### "Q" = 0.5
### "node_dim" = 10
### "num_walks" = 30
### "walk_length" = 10

In [20]:
train_file = "data/train.csv"
valid_file = "data/valid.csv"
test_file = "data/test.csv"

np.random.seed(0)
train_edges = load_data(train_file)
graph = construct_graph_from_edges(train_edges)
valid_edges = load_data(valid_file)
false_edges = generate_false_edges(train_edges+valid_edges, 40000-len(valid_edges))
test_edges = load_test_data(test_file)

number of nodes: 8328
number of edges: 100000
8474 119268


In [21]:
alias_nodes, alias_edges = preprocess_transition_probs(graph, p=1, q=0.5)

In [23]:
model = build_deepwalk(graph, alias_nodes, node_dim=10, num_walks=30, walk_length=10)
scores = [get_cosine_sim(model, src, dst) for src, dst in test_edges]
write_pred("pred.csv", test_edges, scores)

building a DeepWalk model...	number of walks: 249840	average walk length: 9.8983	training time: 41.2719
